# Custom Circuits
With the `Circuit` class one can define and analyze custom circuits. The `Circuit` and `SymbolicCircuit` classes work hand in hand to **identify the periodic and extended degrees of freedom, and to eliminate the non-dynamical cyclic and frozen modes**. With this, the symbolic expression of the Hamiltonian is generated in terms of an appropriate choice of variables. 

The `Circuit` class also performs the **numerical diagonalization of the circuit Hamiltonian. Hierarchical diagonalization can be enabled for better runtime/memory perfomance.**

## Defining a custom circuit
Any superconducting circuit consists of **capacitances**, **inductances**, and **Josephson junctions**. 

<div class="alert alert-info">
    
Custom circuit definition
    
A custom circuit is specified via its graph consisting of nodes and branches. Start by labeling all circuit nodes with integer numbers n=1,2,3,... Every branch is a circuit element connecting two nodes.
    
For each branch:
   
- specify branch type: `JJ`, `L`, `C` for Josephson junction, inductance, and capacitance
- give the labels of two nodes connected by the branch
- provide circuit-element parameters (EJ and ECJ, EL, and EC, respectively)
</div>


As a concrete and familiar example, consider the circuit of the zero-pi qubit (nodes already labeled):
   
![zeropi](./zeropi-circ.jpg)

   
The graph of any custom circuit is stored in simple YAML format using the syntax illustrated here:

In [ ]:
zp_yaml = """# zero-pi circuit
nodes: 4
branches:
- ["JJ", 1, 2, EJ=10, 20]
- ["JJ", 3, 4, 10, 20]
- ["L", 2, 3, 0.008]
- ["L", 4, 1, 0.008]
- ["C", 1, 3, 0.02]
- ["C", 2, 4, 0.02]
"""

Alternatively, circuit specifications can be stored and loaded as `yaml` files with the same syntax. The graph description of the circuit is needed for creating an instance of the `Circuit` class. 


### More on syntax for entering custom circuits

The example above illustrates most of the syntax rules to be followed. Each branch is represented by 

```"<branch-type>", <node_1>, <node_2>, <param-1> [, <param-2>]```

**Branch types and parameters:**

- `C`:  branch parameter is the charging energy $E_C = \frac{e^2}{2C}$
- `L`:  branch parameter is the inductive energy $E_L = \frac{\Phi_0^2}{(2\pi)^2 L}$
- `JJ`: branch parameters are the Josephson energy $E_J$ and junction charging energy $E_{CJ}$

*Example:* `"C", 1, 3, 0.02` is a capacitance connecting nodes 1 and node 3, with charging energy 0.02 GHz.

**Symbolic vs. numerical branch parameters:**

- Branch parameters can be provided as float values, using the energy units set globally (default: GHz)
- A symbol name can be specified along with the value (e.g., `EJ = 10`). Where appropriate, symbolic expressions are given in terms of such provided symbol names.

    
**Ground node:**

A physical ground node (if to be included in the circuit), always has the label 0. It does not count towards the total number of nodes. For instance, when there are $N$ physical nodes in the circuit, and one of them is grounded, the correct input for the number of nodes should be `nodes: <N-1>`.

## Creating a `Circuit` object
Using the above string defining the zero-pi qubit, we can easily create a `Circuit` object:

In [ ]:
import scqubits as scq
zero_pi = scq.Circuit.from_yaml(zp_yaml, from_file=False, ext_basis="discretized")

Here, `ext_basis` can be set to `"discretized"` or `"harmonic"`, and corresponds to the choice of using "spatial" discretization or decomposition in the harmonic oscillator basis for extended degrees of freedom.


This creation of a `Circuit` automatically runs methods for circuit analysis, quantization, and construction of the circuit Hamiltonian matrix. For instance, we can directly access the **symbolic expression of the circuit Hamiltonian**:

In [ ]:
zero_pi.sym_hamiltonian()

All generalized coordinates are denoted by $\theta_i$; the conjugate charges are given by $Q_i$ for extended degrees of freedom, and by $n_i$ for periodic degrees of freedom. Offset charges are denoted by $n_{gi}$ and external fluxes are denoted by $\Phi_i$. A detailed description of external fluxes and offset charges can be found in [External magnetic flux, offset charges](./custom_circuit.ipynb#External-magnetic-flux,-offset-charges).

.. note:
   The coordinates chosen here generally differ from the node variables. In their construction, periodic and extended 
   degrees of freedom are identified and separated. Furthermore, variable elimination is implemented for cyclic and 
   frozen degrees of freedom (if applicable).
   

The **symbolic Lagrangian in terms of node variables** can be accessed via 

In [ ]:
zero_pi.sym_lagrangian(vars_type="node")

The equivalent expression of the **Lagrangian in terms of the transformed variables** used in `sym_hamiltonian` is:

In [ ]:
zero_pi.sym_lagrangian(vars_type="new")

The classification of the different variables is recorded in `var_categories`:

In [ ]:
zero_pi.var_categories

The transformation matrix which maps the new variables ($\theta_i$) to the node variables ($\varphi_i$) can be inspected through `transformation_matrix`.

In [ ]:
zero_pi.transformation_matrix

Alternatively, these mappings can be represented as equations via

In [ ]:
zero_pi.variable_transformation()

Each variable index comes with a **cutoff for basis truncation**. A list of the attribute names of these cutoffs can be accessed using:

In [ ]:
zero_pi.cutoff_names

We must next set these cutoffs to suitable values.

<div class="alert alert-warning">
    Convergence checks required
    
    Setting appropriate cutoff values and confirming convergence is the user's responsibility!
</div>

In [ ]:
zero_pi.cutoff_n_1 = 5
zero_pi.cutoff_phi_2 = 10
zero_pi.cutoff_phi_3 = 10

Now, we can call `eigenvals()` to obtain **low-lying eigenenergies of the circuit Hamiltonian**: 

In [ ]:
zero_pi.eigenvals()

In [ ]:
zero_pi.cutoff_n_1

Increasing the above cutoff values reveals that these eigenvalues have not converged yet. Increasing cutoff values increases the Hilbert space dimension and, thus, increases memory requirements and runtime. A strategy that can help mitigate this problem is the use of hierarchical diagonalization.

## Hierarchical diagonalization

For a large circuit with many degrees of freedoms, a possibly efficient way of obtaining its low-lying eigenenergies and eigenstates is to partition the system into several subsystems, and use the low-lying energy eigenstates of each subsystems as basis states to diagonalize the full system Hamiltonian. The schematic diagram below illustrates how hierarchical diagonalization is performed for a system by partitioning it into two subsystems:

![HD](./custom_circuit_HD.svg)

For the example zero-pi qubit system, the expression from `sym_hamiltonian` shows that $\theta_2$ corresponds to the harmonic zeta mode of the zero-pi qubit. The remaining variables $\theta_1$ and $\theta_3$ form the primary qubit degrees of freedom and may be considered a "separate", weakly coupled subsystem. 

This idea of a **subsystem hierarchy is made explicit by grouping circuit variable indices in a nested list**:

In [ ]:
system_hierarchy = [[1,3], [2]]

This nested list groups variables $1$ and $3$ into one subsystem, and makes variable $2$ a separate subsystem. 

List nesting extends to multiple layers, so that more complex hierarchies can be captured.  
For example, a zero-pi qubit coupled to an oscillator (variable $4$) could be associated with the hierarchy `[[[1,3], [2]], [4]]`.

For convenience, a default list of truncated Hilbert space dimensions is generated by `truncation_template`:

In [ ]:
from scqubits.core.circuit import truncation_template
truncation_template(system_hierarchy)

This default is meant to provide a list of the right shape whose entries should of course be adjusted. 

**To enable hierarchical diagonalization, the system hierarchy and truncation scheme info are handed over to** `set_system_hierarchy`:

In [ ]:
zero_pi.set_system_hierarchy(system_hierarchy=system_hierarchy, subsystem_trunc_dims=[150, 80])

Once the hierarchy is set, subsystem Hamiltonians can be viewed via

In [ ]:
zero_pi.sym_hamiltonian(subsystem_index=0)  # show Hamiltonian for subsystem 0

Hamiltonian terms describing the coupling between two subsystems are displayed via

In [ ]:
zero_pi.sym_interaction((0,1))  # show coupling terms between subsystems 0 and 1

(For the symmetric zero-pi qubit, the zeta mode and the primary qubit degrees of freedom decouple.)

Each subsystem has access to circuit methods, like `eigenvals`. Here are the unconverged (!) eigenenergies for the zeta mode:

In [ ]:
zero_pi.subsystems[1].eigenvals()

Hierarchical diagonalization allows us to increase variable-specific cutoffs without exploding the dimension of the joint Hilbert space.

In [ ]:
zero_pi.cutoff_n_1 = 10
zero_pi.cutoff_ext_2 = 100
zero_pi.cutoff_ext_3 = 50

zero_pi.subsystems[1].eigenvals()

## External magnetic flux, offset charges

All external magnetic fluxes and offset charges are set to zero by default; these can be set to other values. 

Each offset charge $n_{gi}$ is associated with a variable $\theta_i$. Only periodic variables have adjustable associated offset charges; offset charges of extended variables can always be eliminated from Hamiltonian by suitable gauge transformations. A list of adjustable offset charge variables and their maps with node offset charges $\{q_{g\mu}\}$ is provided by:

In [ ]:
zero_pi.offset_charge_transformation()

Each external flux $\Phi_i$ is associated with a branch in a closed loop of circuit. All the external fluxes, with their associated branches and loops (provided as a list of branches), can be accessed via:

In [ ]:
zero_pi.sym_external_fluxes()

External fluxes (in unit of $2\pi$) and offset charges can be adjusted by simply:

In [ ]:
zero_pi.Φ1 = 0.5
zero_pi.ng_1 = 0.6

## Visualization
### Simple parameter sweeps
Plotting the energy spectrum with varying one of the circuit parameters works just as for other qubit classes. For example:

In [ ]:
import numpy as np
# zero_pi.plot_evals_vs_paramvals("Φ1", np.linspace(0,1,21))

<div class="alert alert-warning">
    Work in progress
    
    Need to wait for Sai to push his changes on fixing `I_operator` issue.
</div>


### Plotting the potential
Potential of the circuit can also be plotted using the method `plot_potential`. To see how the potential expression looks like, we can call the attribute `potential_symbolic`:

In [ ]:
zero_pi.potential_symbolic

There are three degrees of freedoms $(\theta_1, \theta_2, \theta_3)$ in this potential for which ranges or values need to be specified. Plot ranges for at most two variables can be specified; values need to be specified for all the other variables.

In [ ]:
zero_pi.plot_potential(θ1=np.linspace(-np.pi, np.pi), θ3=np.linspace(-10*np.pi, 10*np.pi), θ2 = 0.)

`plot_potential` returns a line plot when only one variable is specified with a range:

In [ ]:
zero_pi.plot_potential(θ1=np.linspace(-np.pi, np.pi), θ3=0, θ2 = 0)

### Plotting the wavefunction
Wavefunctions can be plotted with `plot_wavefunction` method. It takes two arguments: `n` specifies the energy eigenstate for which the wavefunction is plotted, `var_indices` specifies variables along which the wavefunction needs to be plotted:

In [ ]:
zero_pi.plot_wavefunction(which=0, var_indices=(1,))

<div class="alert alert-warning">
    Work in progress
    
    Need to wait for Sai to push his changes on fixing `I_operator` issue.
</div>

A maximum of two variable indices can be specified for a plot:

In [ ]:
zero_pi.plot_wavefunction(which=0, var_indices=(1,2))

## Extra features

### Regroup external fluxes
Grouping of external flux variables with specific circuit branches is arbitrary in the case of time-independent flux. The user can provide the list of branches with which external fluxes are grouped. (Splitting flux between multiple branches is not supported.)

In [ ]:
zero_pi.branches

In [ ]:
closure_branches = [zero_pi.branches[3]]
zero_pi.set_closure_branches(closure_branches)

### Customize variable transformations
It is possible to carry out variable transformations with an user-defined transformation matrix. For example, if we want to work with a more-commonly-seen set of variables for zero-pi circuit:

In [ ]:
zero_pi.Φ1 = 0.5
zero_pi.ng_1 = 0.6

In [ ]:
import numpy as np
trans_mat = np.array([[ -1,  -1,  1,  1],
                       [ 1,  1,  1,  1],
                       [ 1,  -1, -1,  1],
                       [ -1,  1,  -1,  1]])*0.5
zero_pi.configure(transformation_matrix=trans_mat)
zero_pi.variable_transformation()

In [ ]:
zero_pi.sym_hamiltonian()

In [ ]:
zero_pi.var_categories

Notice that all the cutoffs, information about system hierarchy, external fluxes and offset charges are restored to default values when `configure` is called. Therefore, user need to set these attributes:

<div class="alert alert-warning">
    Work in progress
    
    Still waiting for Sai to decide what do `configure` reset
</div>

In [ ]:
zero_pi.cutoff_names

In [ ]:
zero_pi.cutoff_n_1 = 20
zero_pi.cutoff_ext_2 = 30
zero_pi.cutoff_ext_3 = 40
system_hierarchy = [[1,3], [2]]
zero_pi.set_system_hierarchy(system_hierarchy=system_hierarchy, subsystem_trunc_dims=[20, 20])

<div class="alert alert-warning">
    Use only `initiate_circuit` to update transformation matrix
    
    Call `zero_pi.transformation_matrix = trans_mat` will not update the variable transformations.
</div>

Regrouping external fluxes, specifying custom variable transformations and setting system hierarchy can all be done with a single call of `initiate_circuit`:

In [ ]:
zero_pi.initiate_circuit(transformation_matrix=trans_mat, 
                         system_hierarchy=system_hierarchy, 
                         subsystem_trunc_dims=[50, 100], 
                         closure_branches=zero_pi.closure_branches)

## Tips

### Options for automatic variable transformation
- After the periodic, frozen and cyclic variables are identified, transformation matrix is completed by taking a set of linearly-independent basis as a reference. The basis set is encoded in a matrix. An identity matrix is used as the reference by default. However, sometimes user may not find the resulting variable transformation convenient. As an alternative, heuristic-based method for generating the basis matrix is available by setting the optional argument `basis_completion="simple"` when creating the `Circuit` object (the default is `"standard"`)

<div class="alert alert-warning">
    Work in progress
    
    Need Sai to clarify the options for automatic variable transformation. Also need to rename `"standard"` as `"canonical basis vector"`
</div>

### Handling large circuits
- When the circuit contains more than 4 nodes, it is better to not use many symbols for capacitances in the input file as the symbolic matrix conversion required for symbolic Hamiltonian can be expensive. This functionality will be improved in the future.
- For large circuits, as the Hilbert space dimension can grow up very fast by increasing cutoffs, numerical diagonalizations with large cutoffs would be challenging in terms of runtime and memory. It is recommended to start diagonalizations with small cutoffs, and gradually increase them as needed. This applies to both the direct and hierarchical diagonalization methods.
- For hierarchical diagonalizations: as diagonalizations for systems with 3 or more variables take much longer time than those for a smaller system with 1 or 2 nodes, it is recommended to group less than 3 nodes for each subsystem, and build a hierarchy of subsystems.

<div class="alert alert-warning">
    Work in progress
    
    Add more tips
</div>